# Inferring Cluster Model Parameters from an X-ray Image

Xray Image dataset. Cartoon cluster model. Revisit dataset. PGM for forward model. Mock data.

Uncertainty encoded by probability distributions. Sampling distribution, likelihood.
Writing down likelihood functions based on assumptions about the data. Independent data points (uncorrelated pixels). "Prior over datasets."

Conditional dependence of data on parameters. Posterior PDF as the result of a statistical inference. Be able to derive the posterior PDF given a likelihood and a prior, and understand what the prior PDF represents and what is involved in assigning it.

Revive PGMs as illustration tool. PGMs as representations of *joint PDFs.* 
Show factorisation, discuss meaning of posterior PDF. 

Compute posterior PDF on 2D grid. Normalize, contours enclosing probability.

Relationship between posterior and likelihood, priors. Learning.  